In [1]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [2]:
pop_2010_2019 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TN/pop_est_2010-2019_tn.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Anderson,75299,75157,75456,75528,76056,76287,76978
1,Bedford,45565,46251,46948,47442,48211,49146,49713
2,Benton,16360,16197,16220,16082,15993,16246,16160
3,Bledsoe,13910,14501,14601,14738,14895,14883,15064
4,Blount,124874,125847,126954,128264,129999,131331,133088


In [3]:
pop_2020_2021 = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TN/pop_est_2020-2021_tn.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Anderson,77209,77576
1,Bedford,50302,51119
2,Benton,15812,15872
3,Bledsoe,14930,15234
4,Blount,135566,137605


In [4]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined.drop(index=pop_combined.index[-1],axis=0,inplace=True)
pop_combined.head()

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Anderson,75299,75157,75456,75528,76056,76287,76978,77209,77576
1,Bedford,45565,46251,46948,47442,48211,49146,49713,50302,51119
2,Benton,16360,16197,16220,16082,15993,16246,16160,15812,15872
3,Bledsoe,13910,14501,14601,14738,14895,14883,15064,14930,15234
4,Blount,124874,125847,126954,128264,129999,131331,133088,135566,137605


In [5]:
metros = pd.read_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_tn = metros_df[metros_df['State Name']=='Tennessee']
metros_tn.head()

,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
4,"Knoxville, TN",Metropolitan Statistical Area,Anderson,Tennessee
22,"Shelbyville, TN",Micropolitan Statistical Area,Bedford,Tennessee
28,"Knoxville, TN",Metropolitan Statistical Area,Blount,Tennessee
30,"Cleveland, TN",Metropolitan Statistical Area,Bradley,Tennessee
55,"Knoxville, TN",Metropolitan Statistical Area,Campbell,Tennessee


In [6]:
file ='C:/Users/iulia/Documents/NSS/projects/capstone/data/raw_data/TN/TN_2010_2022.pdf'

In [7]:
pdf = pdfplumber.open(file)
table = pdf.pages[43].extract_table()
text = pdf.pages[43].extract_text()
table = table [1:]
last_ele = table[-1]
for ele in last_ele:
     match = re.search("(Total\s\s\d{4})",ele)
     boolean = bool(match)
     year = re.findall("(?<=Total\s\s)\d{4}",ele)
     if boolean ==True:
        print(year,"Found!")

['2020'] Found!


### Extracting data for years 2013 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [8]:
pages = list(range(12,44))
tn_df = pd.DataFrame(columns=['County','fatalities','injuries','PDO','total_crashes'])
pdf = pdfplumber.open(file)
for page in pages:            
    tbl = pdf.pages[page].extract_table()
    last_ele = tbl[-1]
    df = pd.DataFrame(tbl[1::],columns=['County','fatalities','injuries','PDO','total_crashes'])
    tn_df = tn_df.append(df)    
    for ele in last_ele:
        match = re.search("(Total\s\s\d{4})",ele)
        boolean = bool(match)
        year = re.findall("(?<=Total\s\s)\d{4}",ele)
        for var in year:
             name = var         
        if boolean == True:
            name = int(name)
            tn_df = tn_df.drop(['PDO'],axis=1)
            tn_df = tn_df.iloc[:-1,:]
            df_combined_tn = pd.merge(tn_df,pop_combined[['County', name]],on="County",how='left')
            df_combined_tn_new = pd.merge(df_combined_tn,metros_tn[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
            df_combined_tn_new.to_excel('C:/Users/iulia/Documents/NSS/projects/capstone/data/cleaned_data/%s_TN.xlsx' % name, engine='xlsxwriter',index=False)
            tn_df = pd.DataFrame(columns=['County','fatalities','injuries','PDO','total_crashes'])
        else :
            continue